Inspired by Deep Learning for Computer Vision with Python [Rosebrock]  
Chapter 12  
ShallowNet: Animals

In [ ]:
from os.path import expanduser
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from imutils import paths
from dlcv_utils import SimplePreprocessor, ImageToArrayPreprocessor, SimpleDatasetLoader

In [ ]:
# path to input dataset
DATASET = expanduser('~/dlcv/code/datasets/animals')

# get the list of image paths
image_paths = list(paths.list_images(DATASET))
if not image_paths:
    print('Error! No files found!')
    quit()

# initialize the image preprocessors
sp = SimplePreprocessor(32, 32)
iap = ImageToArrayPreprocessor()

# load the dataset from disk
sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(image_paths, verbose=500)
print('data shape', data.shape)

# split the data into training (75%) and testing (25%) sets
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# scale data to the range of [0, 1]
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
# build the model
width = 32
height = 32
depth = 3
classes = 3

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(height, width, depth))) # input layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(classes))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

# initialize the gradient descent optimizer
sgd = tf.keras.optimizers.SGD(0.005)

# compile the model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# train the model
EPOCHS = 100
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=32, epochs=EPOCHS, verbose=1)

# save the model to disk
model.save('shallownet_weights.keras')

In [ ]:
# evaluate the network
predictions = model.predict(testX, batch_size=32)
label_names = ['cat', 'dog', 'panda']
print(label_names)
report = classification_report(
    y_true=testY.argmax(axis=1),
    y_pred=predictions.argmax(axis=1),
    target_names=label_names)
print(report)

# plot the training loss and accuracy
plt.style.use('ggplot')
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history['loss'], label='train_loss')
plt.plot(np.arange(0, EPOCHS), H.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, EPOCHS), H.history['accuracy'], label='train_acc')
plt.plot(np.arange(0, EPOCHS), H.history['val_accuracy'], label='val_acc')
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.show()